In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\pc\anaconda3\envs\DS_chatbot_env\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [23]:
import os
from dotenv import load_dotenv

In [24]:
load_dotenv()

True

In [25]:
PINECONE_API_KEY = os.getenv(PINECONE_API_KEY)
PINECONE_API_ENV = os.getenv(PINECONE_API_ENV)

In [5]:
# Extract Data from PDF
def load_pdf(data):
    loader = DirectoryLoader(data, 
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
extracted_Data = load_pdf("data/")

In [7]:
# extracted_Data

In [8]:
# Create text chunks
def text_split(extracted_Data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_Data)
    return text_chunks

In [9]:
text_chunks = text_split(extracted_Data)
print("Length of my chunk: ", len(text_chunks))

Length of my chunk:  1646


In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [13]:
sample_query = embeddings.embed_query("Hello There")
print("Length of sample query: ", len(sample_query))

Length of sample query:  384


In [14]:
# initialize Pinecone

pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV
)
# give your own pine index name
index_name = "datasciencegpt"

# createing embeddings for each of the text chunks & store
docsearch = Pinecone.from_texts(
    [t.page_content for t in text_chunks], embeddings, index_name=index_name
)

In [15]:
# Let us get top k results for our query
doc_search = Pinecone.from_existing_index(index_name, embeddings)
query = "Tell me what is Machine Learning?"
docs = doc_search.similarity_search(query, k=3)
print("Result, ", docs)


Result,  [Document(page_content='definition of term , What Is Machine Learning?\nfeature extraction and selection , Feature Extraction and Selection\nmodeling , Modeling\noverfitting and underfitting , Overfitting and Underfitting\nresources for learning about , For Further Exploration\nmagnitude, computing , V ectors\nmanipulating data , Manipulating Data\nMapReduce\nanalyzing status updates example , Example: Analyzing Status Updates\nbasic algorithm , MapReduce', metadata={}), Document(page_content='reinforcement models , What Is Machine Learning?\nrelational databases , Databases and SQL\nrequests library , HTML and the Parsing Thereof\nrescaling data , Rescaling\nrobots.txt files , Example: Keeping T abs on Congress\nS\nscalar multiplication , V ectors\nscale , Rescaling\nscatterplot matrix , Many Dimensions\nscatterplots , Scatterplots - For Further Exploration\nscikit-learn , For Further Exploration , For Further Exploration , For Further', metadata={}), Document(page_content='p

In [16]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [17]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [19]:
llm=CTransformers(model="model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [20]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [22]:
user_input = input(f"Ask a Question related to Data Science")
result = qa({"query":user_input})
print("Response : ", result["result"])

Response :  Machine learning is a subset of artificial intelligence (AI) that involves using algorithms to analyze and learn patterns in data, and make predictions or decisions based on that data. It involves training computer models on large datasets, so they can make accurate predictions or decisions without being explicitly programmed to do so.
